Copyright 2024 International Meteorological Consultant Inc.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT 
LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. 
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, 
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE 
SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Overview

This jupyter notebook contains the code to read iris raw data files produced by the cox's bazar, kepupara and moulvibazar radars, and convert the data into the netcdf4 format, by using the wradlib library.

[https://docs.wradlib.org/en/latest/index.html]


### Import required libraries including the wradlib.

In [ ]:
import time
import math
import os

import xarray as xr
import wradlib # Tested with wradlib version 2.0.3
import numpy as np
from cftime import date2num
from netCDF4 import Dataset #pylint: disable=no-name-in-module

Set some constant values

In [ ]:
# Earth Radius at the equator and at the pole in meter.
EQUATORIAL_RADIUS = 6378.1370 * 1000
POLAR_RADIUS = 6356.7523 * 1000
INSTITUTION = "Bangladesh Meteorological Department"
# INTERPOLATION = wradlib.ipol.Nearest
INTERPOLATION = wradlib.ipol.Idw

### init

Please set the following variables.

In [ ]:
iris_raw_data_file = "./MLV_raw_201303/MLV130322030018.RAWHUKN" # IRIS file path to be opened.
output_file_path = "./mlv_notebook_test.nc" # netcdf file path to be created.
output_unit = "mmh" # either "mmh" (mm/h rain intensity) or "dbz" (radar reflectivity)
image_width = 360
image_height = 360

Open the set iris raw file and read the contents by using read_iris function of the wradlib.

In [ ]:
with open(iris_raw_data_file, mode="rb") as f:
  iris_content = wradlib.io.read_iris(f, debug=False)

# Get the file name of the given iris file path. This is used as a metadata in the netcdf file later.
file_name = os.path.basename(iris_raw_data_file)

### __read_iris_header_part

Read the header part of the IRIS raw data in the iris_content variable.

In [ ]:
product_hdr = iris_content['product_hdr']
product_config = iris_content['product_hdr']['product_configuration']
product_type = iris_content['product_type']
if product_type != 'RAW':
  raise ValueError("This IRIS file is not a RAW data file. It is not supported.")

beta = product_config['zr_exponent'] / 1000
B = product_config['zr_constant'] / 1000

# generation_time as a datetime object.
generation_time = product_config['generation_time']
# product name looks like 'PPI_A_Z   '. Watch out the trailing spaces.
product_name = product_config['product_name']

# task name looks like 'PPI_A     '. Watch out the trailing spaces.
task_name = product_config['task_name']

product_end = product_hdr['product_end']
radar_lat = product_end['latitude']
radar_lon = product_end['longitude']
center = {'lat': radar_lat, 'lon': radar_lon}
ground_height = product_end['ground_height']
radar_height = product_end['radar_height']
radar_site_name = product_end['site_name']
radar_number_bins = product_end['number_bins']
last_bin_range  = product_end['last_bin_range']
radius = last_bin_range / 100 # It is stored in cm. Convert it to meter.

### __read_iris_data_part

Read the data part of the IRIS raw data in the iris_content variable.

In [ ]:
sweep_data = iris_content['data'][1]['sweep_data']

# DB_DBZ is a ndarray of shape (360, 352). 360 degrees x 352 range bins.
dbz_polar = sweep_data['DB_DBZ']

# azimuth is of shape (360,) and contains the azimuth angles in degrees.
azimuth = sweep_data['azimuth']

# the array of elevations. The array size is 360.
elevation = sweep_data['elevation']

# Number of range bins for each azimuth. rbins is of shape (360, ). It contains 352 for all az.
rbins = sweep_data['rbins']

# Calculate the bin size
bin_size = radius / rbins[0]

### polar to cartesian

Georeference and reproject the data into EPSG:4326

In [ ]:
data = dbz_polar
radar_location = (center['lon'], center['lat'], ground_height + radar_height)
da = wradlib.georef.create_xarray_dataarray(
    data,
    r=np.arange(bin_size / 2, data.shape[1] * bin_size + bin_size / 2, bin_size),
    phi=azimuth,
    theta=elevation,
    site=radar_location,
    sweep_mode="azimuth_surveillance",
)
da.wrl.georef.georeference()
# Reproject the data into EPSG:4326
data_array = da.wrl.georef.reproject(trg_crs=wradlib.georef.epsg_to_osr(4326))

### transform

Convert the radar reflectivity into rain intensity

In [ ]:
if output_unit == 'mmh':
  # Convert the radar reflectivity from dbz to Z(mm6/m3)
  Z = data_array.wrl.trafo.idecibel()
  # Calculate the rain intensity from Z(mm6/m3) ot R(mm/h)
  R = Z.wrl.zr.z_to_r(a=B, b=beta)
  data_array.values = R
  display("min R = %f, max R = %f" % (R.min(), R.max()))
elif output_unit == 'dbz':
  pass
else:
  raise ValueError(f"Unsupported output_unit {output_unit}")

data_array.plot(x="x", y="y")

### output_netcdf

Transform the radial data into grid data.

In [ ]:
out = output_file_path
nc = Dataset(out, "w", format="NETCDF4")
nc.title = file_name
nc.history = "Created " + time.ctime(time.time())
nc.Conventions = 'CF-1.6'
nc.institution = INSTITUTION

# Distance per 1-degree latitude
lat_dist = (2 * math.pi * POLAR_RADIUS) / 360
lon_dist = (2 * math.pi * EQUATORIAL_RADIUS) / 360

# Create the grid data.
lat_array = np.linspace(center['lat'] - radius / lat_dist, center['lat'] + radius / lat_dist, image_height)
lon_array = np.linspace(center['lon'] - radius / lon_dist, center['lon'] + radius / lon_dist, image_width)
cart = xr.Dataset(coords={"x": (["x"], lon_array), "y": (["y"], lat_array)})
grid_data = data_array.wrl.comp.togrid(
  cart, radius=radius, center=(center['lon'], center['lat']), interpol=INTERPOLATION
)
grid_data.plot()

Create the netcdf file.

In [ ]:
# Create dimensions
nc.createDimension("time", 1)
nc.createDimension("lat", len(lat_array))
nc.createDimension("lon", len(lon_array))

# Create variables (time, lat, lon, precipitation_rate)
times = nc.createVariable("time","f4",("time",))
times.units = "hours since 0001-01-01 00:00:00.0"
times.calendar = "standard"
dates = [generation_time]
times[:] = date2num(dates,units=times.units,calendar=times.calendar)

latitudes = nc.createVariable("lat","f4",("lat",))
latitudes.units = "degrees_north"
latitudes.long_name = "latitude"
latitudes[:] = lat_array

longitudes = nc.createVariable("lon","f4",("lon",))
longitudes.units = "degrees_east"
longitudes.long_name = "longitude"
longitudes[:] = lon_array

precipitation_rate = nc.createVariable("precipitation_rate","f4",("time", "lat", "lon"))
precipitation_rate.units = "mm/h" if output_unit == "mmh" else "dbz"
precipitation_rate.long_name = "precipitation rate"
precipitation_rate[0,:,:] = grid_data.values

display("min R = %f, max R = %f" % (precipitation_rate[0,:,:].min(), precipitation_rate[0,:,:].max()))

nc.close()